# Лабораторна робота No2
## ФБ-21 Хав'юк Андрій
## Наука про дані: підготовчий етап

**Мета роботи**: ознайомитися з основними кроками по роботі з даними –
workflow від постановки задачі до написання пояснювальної записки,
зрозуміти постановку задачі та природу даних, над якими виконується
аналітичні операції

**Основні поняття**: сирі дані (raw data), підготовка даних (data preparation)

1. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. 
Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його
імені потрібно додати дату та час завантаження;

In [1]:
from datetime import datetime
import urllib.request
import os
import pandas as pd

In [15]:
def grabber(i):
    url=f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2020&type=Mean".format(i)
    vhi_url = urllib.request.urlopen(url)
    now = datetime.now()
    date = now.strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"vhi_{i}_{date}.csv"
    with open(filename, "wb") as file:
        file.write(vhi_url.read())

    print(f"VHI for {i} is downloaded...")

for i in range(1,28):
    grabber(i)

VHI for 1 is downloaded...
VHI for 2 is downloaded...
VHI for 3 is downloaded...
VHI for 4 is downloaded...
VHI for 5 is downloaded...
VHI for 6 is downloaded...
VHI for 7 is downloaded...
VHI for 8 is downloaded...
VHI for 9 is downloaded...
VHI for 10 is downloaded...
VHI for 11 is downloaded...
VHI for 12 is downloaded...
VHI for 13 is downloaded...
VHI for 14 is downloaded...
VHI for 15 is downloaded...
VHI for 16 is downloaded...
VHI for 17 is downloaded...
VHI for 18 is downloaded...
VHI for 19 is downloaded...
VHI for 20 is downloaded...
VHI for 21 is downloaded...
VHI for 22 is downloaded...
VHI for 23 is downloaded...
VHI for 24 is downloaded...
VHI for 25 is downloaded...
VHI for 26 is downloaded...
VHI for 27 is downloaded...


2. Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів
тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [18]:
def reader():
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    files = os.listdir()
    dataframe = pd.DataFrame()
    for i, file in enumerate(files):
        path = file
        try:
            df = pd.read_csv(path, header=1, names=headers)
            df = df.drop(df.loc[df["VHI"] == -1].index)
            df["area"] = file.split("/")[-1].split("_")[1]
            df["Year"] = df["Year"].str.replace("<tt><pre>", "")
            df = df[~df['Year'].str.contains('</pre></tt>')]
            df.drop('empty', axis=1, inplace=True)
            dataframe = pd.concat([dataframe, df]).drop_duplicates().reset_index(drop=True)
        except Exception as e:
            print(f"Error reading file {file}: {e}")
    return dataframe

df = reader()
print(df)

Error reading file lab2.ipynb: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2

       Year  Week    SMN     SMT    VCI    TCI    VHI area
0      1982   1.0  0.053  260.31  45.01  39.46  42.23    1
1      1982   2.0  0.054  262.29  46.83  31.75  39.29    1
2      1982   3.0  0.055  263.82  48.13  27.24  37.68    1
3      1982   4.0  0.053  265.33  46.09  23.91  35.00    1
4      1982   5.0  0.050  265.66  41.46  26.65  34.06    1
...     ...   ...    ...     ...    ...    ...    ...  ...
53401  2020  48.0  0.115  269.40  57.48  31.51  44.49   27
53402  2020  49.0  0.102  267.62  52.70  33.89  43.29   27
53403  2020  50.0  0.095  266.71  50.56  30.20  40.38   27
53404  2020  51.0  0.084  265.81  44.95  26.76  35.86   27
53405  2020  52.0  0.077  264.70  42.45  27.64  35.05   27

[53406 rows x 8 columns]


3. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на потрібні.

In [4]:
def id_changer(df):
    cities =  {1: "Cherkasy", 2: "Chernihiv", 3: "Chernivtsi", 4: "Crimea", 
               5: "Dnipropetrovs'k", 6: "Donets'k", 7: "Ivano-Frankivs'k", 8: "Kharkiv", 
               9: "Kherson", 10: "Khmel'nyts'kyy", 11: "Kyiv", 12: "Kiev City", 
               13: "Kirovohrad", 14: "Luhans'k", 15: "L'viv", 16: "Mykolayiv", 
               17: "Odessa", 18: "Poltava", 19: "Rivne", 20: "Sevastopol'", 
               21: "Sumy", 22: "Ternopil'", 23: "Transcarpathia", 24: "Vinnytsya", 
               25: "Volyn", 26: "Zaporizhzhya", 27: "Zhytomyr"}
    df['area'] = df['area'].replace(cities)
    
    return df

df = id_changer(df)
print(df)

      Year  Week    SMN     SMT    VCI    TCI    VHI      area
0     1982   1.0  0.053  260.31  45.01  39.46  42.23  Cherkasy
1     1982   2.0  0.054  262.29  46.83  31.75  39.29  Cherkasy
2     1982   3.0  0.055  263.82  48.13  27.24  37.68  Cherkasy
3     1982   4.0  0.053  265.33  46.09  23.91  35.00  Cherkasy
4     1982   5.0  0.050  265.66  41.46  26.65  34.06  Cherkasy
...    ...   ...    ...     ...    ...    ...    ...       ...
2023  2020  48.0  0.115  269.40  57.48  31.51  44.49  Zhytomyr
2024  2020  49.0  0.102  267.62  52.70  33.89  43.29  Zhytomyr
2025  2020  50.0  0.095  266.71  50.56  30.20  40.38  Zhytomyr
2026  2020  51.0  0.084  265.81  44.95  26.76  35.86  Zhytomyr
2027  2020  52.0  0.077  264.70  42.45  27.64  35.05  Zhytomyr

[53406 rows x 8 columns]


Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):
- Ряд VHI для області за вказаний рік, пошук екстремумів (min та
max);

In [5]:
def max(df, area, year):
    vhi_max = df[(df["area"] == area) & (df["Year"] == year)]["VHI"].max()
    return vhi_max

def min(df, area, year):
    vhi_min = df[(df["area"] == area) & (df["Year"] == year)]["VHI"].min()
    return vhi_min

for area in df["area"].unique():
    vhi_max = max(df, area, "2020")
    print(area, "max value: ", vhi_max)

vhi_min = min(df, "Kyiv", "2020")
print("Min: ", vhi_min)

Cherkasy max value:  60.9
Chernihiv max value:  66.55
Chernivtsi max value:  66.4
Crimea max value:  39.47
Dnipropetrovs'k max value:  68.14
Donets'k max value:  59.14
Ivano-Frankivs'k max value:  62.91
Kharkiv max value:  66.26
Kherson max value:  63.49
Khmel'nyts'kyy max value:  67.65
Kyiv max value:  56.3
Kiev City max value:  60.83
Kirovohrad max value:  63.63
Luhans'k max value:  64.39
L'viv max value:  63.06
Mykolayiv max value:  58.11
Odessa max value:  46.71
Poltava max value:  64.77
Rivne max value:  66.01
Sevastopol' max value:  51.22
Sumy max value:  72.11
Ternopil' max value:  68.55
Transcarpathia max value:  68.67
Vinnytsya max value:  64.12
Volyn max value:  61.93
Zaporizhzhya max value:  66.4
Zhytomyr max value:  58.33
Min:  29.27


Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
- Ряд VHI за всі роки для області, виявити роки з екстремальними посухами, які торкнулися більше вказаного відсотка області;

In [6]:
def drought1(df):
    df_area = df[df["area"] == "Lviv"]
    df_drought = df[(df.VHI <= 10)]["Year"].unique()
    df_drought.sort()
    return df_drought

print(drought1(df))

['1993' '2000' '2007']


Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
- Аналогічно для помірних посух

In [7]:
def drought2(df):
    df_area = df[df["area"] == "Lviv"]
    df_drought = df[(df.VHI >= 10) & (df.VHI <= 20)]["Year"].unique()
    df_drought.sort()
    return df_drought

print(drought2(df))

['1982' '1984' '1985' '1986' '1989' '1990' '1992' '1993' '1994' '1995'
 '1996' '1999' '2000' '2003' '2007' '2009' '2010' '2012' '2014' '2015'
 '2017' '2019' '2020']


виявити роки, протягом яких екстремальні посухи торкнулися
більше вказаного відсотка областей по Україні (20% областей -
5 областей з 25)

In [32]:
def global_extremal_drought(df, percentage):
    result = []

    for year, group_df in df.groupby('Year'):
        total_areas = len(group_df['area'].unique())
        drought_areas = len(group_df[group_df['VHI'] < 15]['area'].unique())

        if (drought_areas / total_areas) > (percentage / 100):
            percentage_str = f"{round((drought_areas / total_areas) * 100, 2)}%"
            result.append((year, drought_areas, percentage_str))

    return result

drought_years_extreme = global_extremal_drought(df, 10)
print(drought_years_extreme)

[('2000', 6, '22.22%'), ('2007', 5, '18.52%')]
